Previously I calculated dN/dS for cell type-specific protein coding genes. Now I want to see whether the high dN/dS values of glia-specific genes or the low dN/dS values of neuron-specific genes are mainly contributed by certain GO term. Here I use GO slim terms for biological processes as well as GO associated with cell size or membrane potential properties. 

In [1]:
import numpy as np
import pandas as pd
import glob

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
pd.set_option('display.max_rows', 10)

# Gene Ontology

In [4]:
# import GO slim terms of all protein-coding genes
go_raw_df = pd.read_csv('../data/GO_slim.tsv',usecols=['TERM','ANNOTATED_GENES'],index_col='TERM',sep='\t')

In [5]:
transposed_go_df = go_raw_df.ANNOTATED_GENES.str.split(',',expand=True).T

In [6]:
go_dfs = []
for i, column in enumerate(transposed_go_df.columns):
    data = {'Gene':transposed_go_df[column]}
    df = pd.DataFrame(data)
    df = df.dropna()
    a_list = df.Gene.values.tolist()
    a_list = [x.strip(' ') for x in a_list]
    a_list_data = {'Gene':a_list}
    df = pd.DataFrame(a_list_data)
    df.insert(1,column,True)
    df.set_index('Gene',inplace=True)
    go_dfs.append(df)

In [7]:
go_dfs[0]

,signal transduction
Gene,
1810024B03Rik,True
2210016L21Rik,True
2700049A03Rik,True
3425401B19Rik,True
4921509C19Rik,True
...,...
Znrf3,True
Zp3,True
Zranb1,True


### On top of GO slim terms, append GO terms for cell size and membrane potential

In [75]:
go_list=['regulation_of_cell_size',
         'membrane_depolarization',
         'membrane_hyperpolarization',
         'membrane_repolarization',
         'regulation_of_membrane_potential']

In [19]:
for GO in go_list:
        df=pd.read_csv('../data/GO_'+GO+'.txt',sep='\t',usecols=[1],index_col=False)
        df.drop_duplicates(inplace=True)
        df[GO]=True
        df.rename(columns={'Symbol':'Gene'},inplace=True)
        df.set_index('Gene',inplace=True)
        go_dfs.append(df)

In [76]:
go_list.extend(go_raw_df.index.to_list())

### Merge GO information with cell type-specific genes and average dN/dS: NEAMO

In [22]:
celltype_list=['neuron','endothelia','astrocyte','microglia','oligodendrocyte']

In [29]:
NEAMO_dict={}
for celltype in celltype_list:
    avg_df=pd.read_csv('../results/mouse.celltype-specific_genes/protein-coding_w_dNdS/avg_93species/'+celltype+'.tsv',
                       sep='\t',index_col=0)
    for df in go_dfs:
        avg_df = pd.merge(avg_df, df, how='left', left_index=True, right_index=True)
        avg_df.replace({df.columns[0]: {np.nan:False}},inplace=True)
    NEAMO_dict[celltype]=avg_df

In [30]:
# Record GO info for celltype-specific genes
with pd.ExcelWriter('../results/NEAMO-specific_GOs.xlsx') as writer:
    for key,df in NEAMO_dict.items():
        df.to_excel(writer,sheet_name=key)
#         writer.save()
    # write to an excel file
    # each sheet is a cell type

In [35]:
NEAMO_dict['neuron']['Neuron-specific']='In Neuron'
NEAMO_dict['endothelia']['Endothelia-specific']='In Endothelia'
NEAMO_dict['astrocyte']['Astrocyte-specific']='In Astrocyte'
NEAMO_dict['microglia']['Microglia-specific']='In Microglia'
NEAMO_dict['oligodendrocyte']['Oligodendrocyte-specific']='In Oligodendrocyte'

In [50]:
# Concatanate NEAMO genes for later contingency analysis
gene_info_df = pd.concat(NEAMO_dict.values(),sort=False)

In [54]:
gene_info_df['Astrocyte-specific'].fillna(value='Not IN Astrocyte',inplace=True)
gene_info_df['Neuron-specific'].fillna(value='Not IN Neuron',inplace=True)
gene_info_df['Microglia-specific'].fillna(value='Not IN Microglia',inplace=True)
gene_info_df['Endothelia-specific'].fillna(value='Not IN Endothelia',inplace=True)
gene_info_df['Oligodendrocyte-specific'].fillna(value='Not IN Oligodendrocyte',inplace=True)

In [57]:
gene_info_df.rename(columns={'mean':'avg_dNdS'},inplace=True)

In [58]:
# Setting the threshold of average dN/dS for a gene to be considered high-dN/dS or low-dN/dS
high_cutoff = gene_info_df['avg_dNdS'].quantile(0.75)
low_cutoff = gene_info_df['avg_dNdS'].quantile(0.25)
high_cutoff
low_cutoff

0.19135320683079413

0.055883309714075666

In [59]:
high_dNdS_df = gene_info_df.loc[(gene_info_df['avg_dNdS'] >= high_cutoff)].copy(deep=True)
low_dNdS_df = gene_info_df.loc[(gene_info_df['avg_dNdS'] <= low_cutoff)].copy(deep=True)
gene_info_df['low_dN/dS'] = 'other genes'
gene_info_df.loc[low_dNdS_df.index,'low_dN/dS'] = '< cutoff'
gene_info_df['high_dN/dS'] = 'other genes'
gene_info_df.loc[high_dNdS_df.index,'high_dN/dS'] = '> cutoff'

In [60]:
med = gene_info_df['avg_dNdS'].median(axis=0)

# Visualization

In [82]:
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sns
import matplotlib.ticker as mtick

In [83]:
matplotlib.rcParams['figure.dpi']= 300

In [84]:
font = {'size':8}
matplotlib.rc('font', **font)

In [81]:
def crosstable (df,Celltype,GO): 
    table = pd.crosstab(df.loc[:,Celltype],df.loc[:,GO])
    odds_ratio = stats.fisher_exact(table)[0]
    chi2stats = stats.chi2_contingency(table)
    col_sum = table.sum(axis=0)
    col_sum.name = 'Total'
    table = table.append(col_sum)
    table['Total'] = table.sum(axis=1)
    # This step can actually be accomplished by stating margins=True in pd.crosstab()
    return(table,chi2stats,odds_ratio)

In [85]:
def tab_plot(fig,df,GO):
    # fig is the figure that this table plot will be draw in
    # N is the serial number of the table. If it's the first table, then input 1
    # df is the dataframe with celltype info columns and GO info columns (here 'gene_info_df')
    # dNdS_cat is the column of category of dN/dS level, so high_dN/dS or low_dN/dS
    
    n_gene_GO = df[df[GO]==True].shape[0]
    
    n=5 # the number of lines ocupied by distribution histogram 
    
    ##### Astrocyte #####
    
    ### low dN/dS ###
    
    # Generating the contingency table for astrocyte and low dN/dS as a dataframe 
    table_A_low, stat_A_low, odds_A_low= crosstable(df[df[GO]==True],'Astrocyte-specific','low_dN/dS')
    # Statistics for Astrocyte and low dN/dS
    tot_low = table_A_low.iloc[2,0] #Total number of genes afiliated with this GO with avg dN/dS lower than 1st quantile
    tot_not_low = table_A_low.iloc[2,1] #Total number of genes afiliated with this GO with avg dN/dS higher than 1st quantile
    tot_A = table_A_low.iloc[0,2] #Total number of astrocyte-specific genes afiliated with this GO
    tot_not_A = table_A_low.iloc[1,2] #Total number of genes afiliated with this GO that are not astrocyte-specific
    # The expected (fitted) value for each cell in the contingency table
    expected_A_low_arr = np.outer(table_A_low.loc[:,'Total'],table_A_low.loc['Total',:])/table_A_low.loc['Total','Total'] 
    # Put the expected value into the table with the observed value 
    table_A_low.iloc[0,0]='{0} ({1:.2f})'.format(table_A_low.iloc[0,0],expected_A_low_arr[0,0])
    table_A_low.iloc[0,1]='{0} ({1:.2f})'.format(table_A_low.iloc[0,1],expected_A_low_arr[0,1])
    table_A_low.iloc[1,0]='{0} ({1:.2f})'.format(table_A_low.iloc[1,0],expected_A_low_arr[1,0])
    table_A_low.iloc[1,1]='{0} ({1:.2f})'.format(table_A_low.iloc[1,1],expected_A_low_arr[1,1])
    # Set axes for contingency table against astrocyte
    tabA_low_plot = fig.add_subplot(grid[n+0,0])
    tabA_low_plot.axis('tight')
    tabA_low_plot.axis('off')
    stringA_low = 'chi2: {0:.3f}, p: {1:.10f}, OR: {2:.5f}'.format(stat_A_low[0],stat_A_low[1],odds_A_low)
    
    ### High dN/dS ### 
    
    # Generating the contingency table for astrocyte and high dN/dS as a dataframe 
    table_A_high, stat_A_high, odds_A_high= crosstable(df[df[GO]==True],'Astrocyte-specific','high_dN/dS')
    # Statistics for Astrocyte and high dN/dS
    tot_high = table_A_high.iloc[2,0]
    tot_not_high = table_A_high.iloc[2,1]
    # The expected (fitted) value for each cell in the contingency table
    expected_A_high_arr = np.outer(table_A_high.loc[:,'Total'],table_A_high.loc['Total',:])/table_A_high.loc['Total','Total'] 
    # Put the expected value into the table with the observed value 
    table_A_high.iloc[0,0]='{0} ({1:.2f})'.format(table_A_high.iloc[0,0],expected_A_high_arr[0,0])
    table_A_high.iloc[0,1]='{0} ({1:.2f})'.format(table_A_high.iloc[0,1],expected_A_high_arr[0,1])
    table_A_high.iloc[1,0]='{0} ({1:.2f})'.format(table_A_high.iloc[1,0],expected_A_high_arr[1,0])
    table_A_high.iloc[1,1]='{0} ({1:.2f})'.format(table_A_high.iloc[1,1],expected_A_high_arr[1,1])
    # Set axes for contingency table against astrocyte
    tabA_high_plot = fig.add_subplot(grid[n+0,1])
    tabA_high_plot.axis('tight')
    tabA_high_plot.axis('off')
    stringA_high = 'chi2: {0:.3f}, p: {1:.10f}, OR: {2:.5f}'.format(stat_A_high[0],stat_A_high[1],odds_A_high)
    
    ##### Microglia #####
    
    ### low dN/dS ###
    
    # Generating the contingency table for microglia and low dN/dS as a dataframe 
    table_M_low, stat_M_low, odds_M_low= crosstable(df[df[GO]==True],'Microglia-specific','low_dN/dS')
    # Statistics for Microglia and low dN/dS
    tot_M = table_M_low.iloc[0,2] #Total number of microglia-specific genes afiliated with this GO
    tot_not_M = table_M_low.iloc[1,2] #Total number of genes afiliated with this GO that are not microglia-specific
    # The expected (fitted) value for each cell in the contingency table
    expected_M_low_arr = np.outer(table_M_low.loc[:,'Total'],table_M_low.loc['Total',:])/table_M_low.loc['Total','Total'] 
    # Put the expected value into the table with the observed value 
    table_M_low.iloc[0,0]='{0} ({1:.2f})'.format(table_M_low.iloc[0,0],expected_M_low_arr[0,0])
    table_M_low.iloc[0,1]='{0} ({1:.2f})'.format(table_M_low.iloc[0,1],expected_M_low_arr[0,1])
    table_M_low.iloc[1,0]='{0} ({1:.2f})'.format(table_M_low.iloc[1,0],expected_M_low_arr[1,0])
    table_M_low.iloc[1,1]='{0} ({1:.2f})'.format(table_M_low.iloc[1,1],expected_M_low_arr[1,1])
    # Set axes for contingency table against microglia
    tabM_low_plot = fig.add_subplot(grid[n+2,0])
    tabM_low_plot.axis('tight')
    tabM_low_plot.axis('off')
    stringM_low = 'chi2: {0:.3f}, p: {1:.10f}, OR: {2:.5f}'.format(stat_M_low[0],stat_M_low[1],odds_M_low)
    
    ### High dN/dS ###
    
    # Generating the contingency table for microglia and high dN/dS as a dataframe 
    table_M_high, stat_M_high, odds_M_high= crosstable(df[df[GO]==True],'Microglia-specific','high_dN/dS')
    # Statistics for Microglia and high dN/dS
    tot_high = table_M_high.iloc[2,0]
    tot_not_high = table_M_high.iloc[2,1]
    # The expected (fitted) value for each cell in the contingency table
    expected_M_high_arr = np.outer(table_M_high.loc[:,'Total'],table_M_high.loc['Total',:])/table_M_high.loc['Total','Total'] 
    # Put the expected value into the table with the observed value 
    table_M_high.iloc[0,0]='{0} ({1:.2f})'.format(table_M_high.iloc[0,0],expected_M_high_arr[0,0])
    table_M_high.iloc[0,1]='{0} ({1:.2f})'.format(table_M_high.iloc[0,1],expected_M_high_arr[0,1])
    table_M_high.iloc[1,0]='{0} ({1:.2f})'.format(table_M_high.iloc[1,0],expected_M_high_arr[1,0])
    table_M_high.iloc[1,1]='{0} ({1:.2f})'.format(table_M_high.iloc[1,1],expected_M_high_arr[1,1])
    # Set axes for contingency table against microglia
    tabM_high_plot = fig.add_subplot(grid[n+2,1])
    tabM_high_plot.axis('tight')
    tabM_high_plot.axis('off')
    stringM_high = 'chi2: {0:.3f}, p: {1:.10f}, OR: {2:.5f}'.format(stat_M_high[0],stat_M_high[1],odds_M_high)
    
    ##### Oligodendrocyte #####
    
    ### low dN/dS ###
    
    # Generating the contingency table for oligodendrocyte and low dN/dS as a dataframe 
    table_O_low, stat_O_low, odds_O_low= crosstable(df[df[GO]==True],'Oligodendrocyte-specific','low_dN/dS')
    # Statistics for Oligodendrocyte and low dN/dS
    tot_O = table_O_low.iloc[0,2] #Total number of oligodendrocyte-specific genes afiliated with this GO
    tot_not_O = table_O_low.iloc[1,2] #Total number of genes afiliated with this GO that are not oligodendrocyte-specific
    # The expected (fitted) value for each cell in the contingency table
    expected_O_low_arr = np.outer(table_O_low.loc[:,'Total'],table_O_low.loc['Total',:])/table_O_low.loc['Total','Total'] 
    # Put the expected value into the table with the observed value 
    table_O_low.iloc[0,0]='{0} ({1:.2f})'.format(table_O_low.iloc[0,0],expected_O_low_arr[0,0])
    table_O_low.iloc[0,1]='{0} ({1:.2f})'.format(table_O_low.iloc[0,1],expected_O_low_arr[0,1])
    table_O_low.iloc[1,0]='{0} ({1:.2f})'.format(table_O_low.iloc[1,0],expected_O_low_arr[1,0])
    table_O_low.iloc[1,1]='{0} ({1:.2f})'.format(table_O_low.iloc[1,1],expected_O_low_arr[1,1])
    # Set axes for contingency table against oligodendrocyte
    tabO_low_plot = fig.add_subplot(grid[n+4,0])
    tabO_low_plot.axis('tight')
    tabO_low_plot.axis('off')
    stringO_low = 'chi2: {0:.3f}, p: {1:.10f}, OR: {2:.5f}'.format(stat_O_low[0],stat_O_low[1],odds_O_low)
    
    ### High dN/dS ###
    
    # Generating the contingency table for oligodendrocyte and high dN/dS as a dataframe 
    table_O_high, stat_O_high, odds_O_high= crosstable(df[df[GO]==True],'Oligodendrocyte-specific','high_dN/dS')
    # Statistics for Oligodendrocyte and high dN/dS
    tot_high = table_O_high.iloc[2,0]
    tot_not_high = table_O_high.iloc[2,1]
    # The expected (fitted) value for each cell in the contingency table
    expected_O_high_arr = np.outer(table_O_high.loc[:,'Total'],table_O_high.loc['Total',:])/table_O_high.loc['Total','Total'] 
    # Put the expected value into the table with the observed value 
    table_O_high.iloc[0,0]='{0} ({1:.2f})'.format(table_O_high.iloc[0,0],expected_O_high_arr[0,0])
    table_O_high.iloc[0,1]='{0} ({1:.2f})'.format(table_O_high.iloc[0,1],expected_O_high_arr[0,1])
    table_O_high.iloc[1,0]='{0} ({1:.2f})'.format(table_O_high.iloc[1,0],expected_O_high_arr[1,0])
    table_O_high.iloc[1,1]='{0} ({1:.2f})'.format(table_O_high.iloc[1,1],expected_O_high_arr[1,1])
    # Set axes for contingency table against oligodendrocyte
    tabO_high_plot = fig.add_subplot(grid[n+4,1])
    tabO_high_plot.axis('tight')
    tabO_high_plot.axis('off')
    stringO_high = 'chi2: {0:.3f}, p: {1:.10f}, OR: {2:.5f}'.format(stat_O_high[0],stat_O_high[1],odds_O_high)
    
        ##### Neuron #####
    
    ### low dN/dS ###
    
    # Generating the contingency table for neuron and low dN/dS as a dataframe 
    table_N_low, stat_N_low, odds_N_low= crosstable(df[df[GO]==True],'Neuron-specific','low_dN/dS')    
    # Statistics for Neuron and low dN/dS
    tot_low = table_N_low.iloc[2,0] #Total number of genes afiliated with this GO with avg dN/dS lower than 1st quantile
    tot_not_low = table_N_low.iloc[2,1] #Total number of genes afiliated with this GO with avg dN/dS higher than 1st quantile
    tot_N = table_N_low.iloc[0,2] #Total number of neuron-specific genes afiliated with this GO
    tot_not_N = table_N_low.iloc[1,2] #Total number of genes afiliated with this GO that are not neuron-specific
    # The expected (fitted) value for each cell in the contingency table
    expected_N_low_arr = np.outer(table_N_low.loc[:,'Total'],table_N_low.loc['Total',:])/table_N_low.loc['Total','Total'] 
    # Put the expected value into the table with the observed value 
    table_N_low.iloc[0,0]='{0} ({1:.2f})'.format(table_N_low.iloc[0,0],expected_N_low_arr[0,0])
    table_N_low.iloc[0,1]='{0} ({1:.2f})'.format(table_N_low.iloc[0,1],expected_N_low_arr[0,1])
    table_N_low.iloc[1,0]='{0} ({1:.2f})'.format(table_N_low.iloc[1,0],expected_N_low_arr[1,0])
    table_N_low.iloc[1,1]='{0} ({1:.2f})'.format(table_N_low.iloc[1,1],expected_N_low_arr[1,1])
    # Set axes for contingency table against neuron
    tabN_low_plot = fig.add_subplot(grid[n+6,0])
    tabN_low_plot.axis('tight')
    tabN_low_plot.axis('off')
    stringN_low = 'n = {0:d} genes belong to {1:s}'.format(n_gene_GO,GO)
    stringN_low = 'chi2: {0:.3f}, p: {1:.10f}, OR: {2:.5f}'.format(stat_N_low[0],stat_N_low[1], odds_N_low)
    
    ### High dN/dS ### 
    
    # Generating the contingency table for neuron and high dN/dS as a dataframe 
    table_N_high, stat_N_high, odds_N_high= crosstable(df[df[GO]==True],'Neuron-specific','high_dN/dS')
    # Statistics for Neuron and high dN/dS
    tot_high = table_N_high.iloc[2,0]
    tot_not_high = table_N_high.iloc[2,1]
    # The expected (fitted) value for each cell in the contingency table
    expected_N_high_arr = np.outer(table_N_high.loc[:,'Total'],table_N_high.loc['Total',:])/table_N_high.loc['Total','Total'] 
    # Put the expected value into the table with the observed value 
    table_N_high.iloc[0,0]='{0} ({1:.2f})'.format(table_N_high.iloc[0,0],expected_N_high_arr[0,0])
    table_N_high.iloc[0,1]='{0} ({1:.2f})'.format(table_N_high.iloc[0,1],expected_N_high_arr[0,1])
    table_N_high.iloc[1,0]='{0} ({1:.2f})'.format(table_N_high.iloc[1,0],expected_N_high_arr[1,0])
    table_N_high.iloc[1,1]='{0} ({1:.2f})'.format(table_N_high.iloc[1,1],expected_N_high_arr[1,1])
    # Set axes for contingency table against neuron
    tabN_high_plot = fig.add_subplot(grid[n+6,1])
    tabN_high_plot.axis('tight')
    tabN_high_plot.axis('off')
    stringN_high = 'chi2: {0:.3f}, p: {1:.10f}, OR: {2:.5f}'.format(stat_N_high[0],stat_N_high[1],odds_N_high)
    
    ##### Endothelia #####
    
    ### low dN/dS ###
    
    # Generating the contingency table for endothelia and low dN/dS as a dataframe 
    table_E_low, stat_E_low, odds_E_low= crosstable(df[df[GO]==True],'Endothelia-specific','low_dN/dS')
    # Statistics for Endothelia and low dN/dS
    tot_E = table_E_low.iloc[0,2] #Total number of endothelia-specific genes afiliated with this GO
    tot_not_E = table_E_low.iloc[1,2] #Total number of genes afiliated with this GO that are not endothelia-specific
    # The expected (fitted) value for each cell in the contingency table
    expected_E_low_arr = np.outer(table_E_low.loc[:,'Total'],table_E_low.loc['Total',:])/table_E_low.loc['Total','Total'] 
    # Put the expected value into the table with the observed value 
    table_E_low.iloc[0,0]='{0} ({1:.2f})'.format(table_E_low.iloc[0,0],expected_E_low_arr[0,0])
    table_E_low.iloc[0,1]='{0} ({1:.2f})'.format(table_E_low.iloc[0,1],expected_E_low_arr[0,1])
    table_E_low.iloc[1,0]='{0} ({1:.2f})'.format(table_E_low.iloc[1,0],expected_E_low_arr[1,0])
    table_E_low.iloc[1,1]='{0} ({1:.2f})'.format(table_E_low.iloc[1,1],expected_E_low_arr[1,1])
    # Set axes for contingency table against endothelia
    tabE_low_plot = fig.add_subplot(grid[n+8,0])
    tabE_low_plot.axis('tight')
    tabE_low_plot.axis('off')
    stringE_low = 'chi2: {0:.3f}, p: {1:.10f}, OR: {2:.5f}'.format(stat_E_low[0],stat_E_low[1],odds_E_low)
    
    ### High dN/dS ###
    
    # Generating the contingency table for endothelia and high dN/dS as a dataframe 
    table_E_high, stat_E_high, odds_E_high= crosstable(df[df[GO]==True],'Endothelia-specific','high_dN/dS')
    # Statistics for Endothelia and high dN/dS
    tot_high = table_E_high.iloc[2,0]
    tot_not_high = table_E_high.iloc[2,1]
    # The expected (fitted) value for each cell in the contingency table
    expected_E_high_arr = np.outer(table_E_high.loc[:,'Total'],table_E_high.loc['Total',:])/table_E_high.loc['Total','Total'] 
    # Put the expected value into the table with the observed value 
    table_E_high.iloc[0,0]='{0} ({1:.2f})'.format(table_E_high.iloc[0,0],expected_E_high_arr[0,0])
    table_E_high.iloc[0,1]='{0} ({1:.2f})'.format(table_E_high.iloc[0,1],expected_E_high_arr[0,1])
    table_E_high.iloc[1,0]='{0} ({1:.2f})'.format(table_E_high.iloc[1,0],expected_E_high_arr[1,0])
    table_E_high.iloc[1,1]='{0} ({1:.2f})'.format(table_E_high.iloc[1,1],expected_E_high_arr[1,1])
    # Set axes for contingency table against endothelia
    tabE_high_plot = fig.add_subplot(grid[n+8,1])
    tabE_high_plot.axis('tight')
    tabE_high_plot.axis('off')
    stringE_high = 'chi2: {0:.3f}, p: {1:.10f}, OR: {2:.5f}'.format(stat_E_high[0],stat_E_high[1],odds_E_high)
    
    
    ########## Drawing Contingency Tables ##########
    
    # Drawing the contingency table against neuron-low onto figure 
    cell_text = []
    for row in range(len(table_N_low)):
        cell_text.append(table_N_low.iloc[row])
    tabN_low = tabN_low_plot.table(cellText=cell_text, rowLabels=table_N_low.index,colLabels=table_N_low.columns, colWidths=[0.35, 0.35, 0.15], loc='center')
    tabN_low.auto_set_font_size(False)
    tabN_low.set_fontsize(7)
    # Drawing the contingency table against neuron-high onto figure
    cell_text = []
    for row in range(len(table_N_high)):
        cell_text.append(table_N_high.iloc[row])
    tabN_high = tabN_high_plot.table(cellText=cell_text, rowLabels=table_N_high.index,colLabels=table_N_high.columns, colWidths=[0.35, 0.35, 0.15], loc='center')
    tabN_high.auto_set_font_size(False)
    tabN_high.set_fontsize(7)
    
    # Drawing the contingency table against endothelia-low onto figure 
    cell_text = []
    for row in range(len(table_E_low)):
        cell_text.append(table_E_low.iloc[row])
    tabE_low = tabE_low_plot.table(cellText=cell_text, rowLabels=table_E_low.index,colLabels=table_E_low.columns, colWidths=[0.35, 0.35, 0.15], loc='center')
    tabE_low.auto_set_font_size(False)
    tabE_low.set_fontsize(7)
    # Drawing the contingency table against endothelia-high onto figure
    cell_text = []
    for row in range(len(table_E_high)):
        cell_text.append(table_E_high.iloc[row])
    tabE_high = tabE_high_plot.table(cellText=cell_text, rowLabels=table_E_high.index,colLabels=table_E_high.columns, colWidths=[0.35, 0.35, 0.15], loc='center')
    tabE_high.auto_set_font_size(False)
    tabE_high.set_fontsize(7)
    
    # Drawing the contingency table against astrocyte-low onto figure 
    cell_text = []
    for row in range(len(table_A_low)):
        cell_text.append(table_A_low.iloc[row])
    tabA_low = tabA_low_plot.table(cellText=cell_text, rowLabels=table_A_low.index,colLabels=table_A_low.columns, colWidths=[0.35, 0.35, 0.15], loc='center')
    tabA_low.auto_set_font_size(False)
    tabA_low.set_fontsize(7)
    # Drawing the contingency table against astrocyte-high onto figure
    cell_text = []
    for row in range(len(table_A_high)):
        cell_text.append(table_A_high.iloc[row])
    tabA_high = tabA_high_plot.table(cellText=cell_text, rowLabels=table_A_high.index,colLabels=table_A_high.columns, colWidths=[0.35, 0.35, 0.15], loc='center')
    tabA_high.auto_set_font_size(False)
    tabA_high.set_fontsize(7)
    
    # Drawing the contingency table against microglia-low onto figure 
    cell_text = []
    for row in range(len(table_M_low)):
        cell_text.append(table_M_low.iloc[row])
    tabM_low = tabM_low_plot.table(cellText=cell_text, rowLabels=table_M_low.index,colLabels=table_M_low.columns, colWidths=[0.35, 0.35, 0.15], loc='center')
    tabM_low.auto_set_font_size(False)
    tabM_low.set_fontsize(7)
    # Drawing the contingency table against microglia-high onto figure
    cell_text = []
    for row in range(len(table_M_high)):
        cell_text.append(table_M_high.iloc[row])
    tabM_high = tabM_high_plot.table(cellText=cell_text, rowLabels=table_M_high.index,colLabels=table_M_high.columns, colWidths=[0.35, 0.35, 0.15], loc='center')
    tabM_high.auto_set_font_size(False)
    tabM_high.set_fontsize(7)
    
    # Drawing the contingency table against oligodendrocyte-low onto figure 
    cell_text = []
    for row in range(len(table_O_low)):
        cell_text.append(table_O_low.iloc[row])
    tabO_low = tabO_low_plot.table(cellText=cell_text, rowLabels=table_O_low.index,colLabels=table_O_low.columns, colWidths=[0.28, 0.28, 0.11], loc='center')
    tabO_low.auto_set_font_size(False)
    tabO_low.set_fontsize(7)
    # Drawing the contingency table against oligodendrocyte-high onto figure
    cell_text = []
    for row in range(len(table_O_high)):
        cell_text.append(table_O_high.iloc[row])
    tabO_high = tabO_high_plot.table(cellText=cell_text, rowLabels=table_O_high.index,colLabels=table_O_high.columns, colWidths=[0.28, 0.28, 0.11], loc='center')
    tabO_high.auto_set_font_size(False)
    tabO_high.set_fontsize(7)
    
    ########## Writing statistics of each contingency table under the table ##########
    
    # Set axes for chi-square statistics for Neuron
    low_Nchi2_plot = fig.add_subplot(grid[n+7,0])
    low_Nchi2_plot.axis('tight')
    low_Nchi2_plot.axis('off')
    high_Nchi2_plot = fig.add_subplot(grid[n+7,1])
    high_Nchi2_plot.axis('tight')
    high_Nchi2_plot.axis('off')
    
    # Set axes for chi-square statistics for Endothilial cells
    low_Echi2_plot = fig.add_subplot(grid[n+9,0])
    low_Echi2_plot.axis('tight')
    low_Echi2_plot.axis('off')
    high_Echi2_plot = fig.add_subplot(grid[n+9,1])
    high_Echi2_plot.axis('tight')
    high_Echi2_plot.axis('off')
    
    # Set axes for chi-square statistics for Astrocyte
    low_Achi2_plot = fig.add_subplot(grid[n+1,0])
    low_Achi2_plot.axis('tight')
    low_Achi2_plot.axis('off')
    high_Achi2_plot = fig.add_subplot(grid[n+1,1])
    high_Achi2_plot.axis('tight')
    high_Achi2_plot.axis('off')
    
    # Set axes for chi-square statistics for Endothilial cells
    low_Mchi2_plot = fig.add_subplot(grid[n+3,0])
    low_Mchi2_plot.axis('tight')
    low_Mchi2_plot.axis('off')
    high_Mchi2_plot = fig.add_subplot(grid[n+3,1])
    high_Mchi2_plot.axis('tight')
    high_Mchi2_plot.axis('off')
    
    # Set axes for chi-square statistics for Oligodendrocytel cells
    low_Ochi2_plot = fig.add_subplot(grid[n+5,0])
    low_Ochi2_plot.axis('tight')
    low_Ochi2_plot.axis('off')
    high_Ochi2_plot = fig.add_subplot(grid[n+5,1])
    high_Ochi2_plot.axis('tight')
    high_Ochi2_plot.axis('off')
    
    low_Achi2_plot.text(0.25, 0.6, stringA_low,
        horizontalalignment='center',
        verticalalignment='center',
        transform=low_Achi2_plot.transAxes)
    high_Achi2_plot.text(0.25, 0.6, stringA_high,
        horizontalalignment='center',
        verticalalignment='center',
        transform=high_Achi2_plot.transAxes)
    
    low_Mchi2_plot.text(0.25, 0.6, stringM_low,
        horizontalalignment='center',
        verticalalignment='center',
        transform=low_Mchi2_plot.transAxes)
    high_Mchi2_plot.text(0.25, 0.6, stringM_high,
        horizontalalignment='center',
        verticalalignment='center',
        transform=high_Mchi2_plot.transAxes)
    
    low_Ochi2_plot.text(0.25, 0.6, stringO_low,
        horizontalalignment='center',
        verticalalignment='center',
        transform=low_Ochi2_plot.transAxes)
    high_Ochi2_plot.text(0.25, 0.6, stringO_high,
        horizontalalignment='center',
        verticalalignment='center',
        transform=high_Ochi2_plot.transAxes)
    
    low_Nchi2_plot.text(0.25, 0.6, stringN_low,
        horizontalalignment='center',
        verticalalignment='center',
        transform=low_Nchi2_plot.transAxes)
    high_Nchi2_plot.text(0.25, 0.6, stringN_high,
        horizontalalignment='center',
        verticalalignment='center',
        transform=high_Nchi2_plot.transAxes)
    
    low_Echi2_plot.text(0.25, 0.6, stringE_low,
        horizontalalignment='center',
        verticalalignment='center',
        transform=low_Echi2_plot.transAxes)
    high_Echi2_plot.text(0.25, 0.6, stringE_high,
        horizontalalignment='center',
        verticalalignment='center',
        transform=high_Echi2_plot.transAxes)
    
    tab_plot.Nchi2_num=stat_N_low[0]
    tab_plot.Np_num=stat_N_low[1]

In [78]:
# create a excel table of contingency analysis stats results for each GO
table_df=pd.DataFrame(columns=['GO','num_genes',
                            'n_med','low_n_chi2','low_n_p',
                            'low_n_OR','low_n_fisher_p',
                            'high_n_chi2','high_n_p',
                            'high_n_OR','high_n_fisher_p',
                            'e_med','low_e_chi2','low_e_p',
                            'low_e_OR','low_e_fisher_p',
                            'high_e_chi2','high_e_p',
                            'high_e_OR','high_e_fisher_p',
                            'o_med','low_o_chi2','low_o_p',
                            'low_o_OR','low_o_fisher_p',
                            'high_o_chi2','high_o_p',
                            'high_o_OR','high_o_fisher_p',
                            'a_med','low_a_chi2','low_a_p',
                            'low_a_OR','low_a_fisher_p',
                            'high_a_chi2','high_a_p',
                            'high_a_OR','high_a_fisher_p',
                            'm_med','low_m_chi2','low_m_p',
                            'low_m_OR','low_m_fisher_p',
                            'high_m_chi2','high_m_p',
                            'high_m_OR','high_m_fisher_p'])
for GO in go_list:
    df=gene_info_df.loc[gene_info_df[GO] == True].copy(deep=True)
    num_genes = df.shape[0]
    try:
        #Neuron
        n_med=df[df['Neuron-specific']=='In Neuron']['avg_dNdS'].median()
        #lower than threshold 
        low_n_table=pd.crosstab(df.loc[:,'Neuron-specific'],df.loc[:,'low_dN/dS'])
        (low_n_OR,low_n_fisher_p)=stats.fisher_exact(low_n_table)
        (low_n_chi2,low_n_p,low_n_dof,low_n_expected_table)=stats.chi2_contingency(low_n_table)
        #higher than threshold
        high_n_table=pd.crosstab(df.loc[:,'Neuron-specific'],df.loc[:,'high_dN/dS'])
        (high_n_OR,high_n_fisher_p)=stats.fisher_exact(high_n_table)
        (high_n_chi2,high_n_p,high_n_dof,high_n_expected_table)=stats.chi2_contingency(high_n_table)
        
        #Endothelia
        e_med=df[df['Endothelia-specific']=='In Endothelia']['avg_dNdS'].median()
        #lower than threshold 
        low_e_table=pd.crosstab(df.loc[:,'Endothelia-specific'],df.loc[:,'low_dN/dS'])
        (low_e_OR,low_e_fisher_p)=stats.fisher_exact(low_e_table)
        (low_e_chi2,low_e_p,low_e_dof,low_e_expected_table)=stats.chi2_contingency(low_e_table)
        #higher than threshold
        high_e_table=pd.crosstab(df.loc[:,'Endothelia-specific'],df.loc[:,'high_dN/dS'])
        (high_e_OR,high_e_fisher_p)=stats.fisher_exact(high_e_table)
        (high_e_chi2,high_e_p,high_e_dof,high_e_expected_table)=stats.chi2_contingency(high_e_table)
        
        #Oligodendrocyte
        o_med=df[df['Oligodendrocyte-specific']=='In Oligodendrocyte']['avg_dNdS'].median()
        #lower than threshold 
        low_o_table=pd.crosstab(df.loc[:,'Oligodendrocyte-specific'],df.loc[:,'low_dN/dS'])
        (low_o_OR,low_o_fisher_p)=stats.fisher_exact(low_o_table)
        (low_o_chi2,low_o_p,low_o_dof,low_o_expected_table)=stats.chi2_contingency(low_o_table)
        #higher than threshold
        high_o_table=pd.crosstab(df.loc[:,'Oligodendrocyte-specific'],df.loc[:,'high_dN/dS'])
        (high_o_OR,high_o_fisher_p)=stats.fisher_exact(high_o_table)
        (high_o_chi2,high_o_p,high_o_dof,high_o_expected_table)=stats.chi2_contingency(high_o_table)
        
        #Astrocyte
        a_med=df[df['Astrocyte-specific']=='In Astrocyte']['avg_dNdS'].median()
        #lower than threshold 
        low_a_table=pd.crosstab(df.loc[:,'Astrocyte-specific'],df.loc[:,'low_dN/dS'])
        (low_a_OR,low_a_fisher_p)=stats.fisher_exact(low_a_table)
        (low_a_chi2,low_a_p,low_a_dof,low_a_expected_table)=stats.chi2_contingency(low_a_table)
        #higher than threshold
        high_a_table=pd.crosstab(df.loc[:,'Astrocyte-specific'],df.loc[:,'high_dN/dS'])
        (high_a_OR,high_a_fisher_p)=stats.fisher_exact(high_a_table)
        (high_a_chi2,high_a_p,high_a_dof,high_a_expected_table)=stats.chi2_contingency(high_a_table)
        
        #Microglia
        m_med=df[df['Microglia-specific']=='In Microglia']['avg_dNdS'].median()
        #lower than threshold 
        low_m_table=pd.crosstab(df.loc[:,'Microglia-specific'],df.loc[:,'low_dN/dS'])
        (low_m_OR,low_m_fisher_p)=stats.fisher_exact(low_m_table)
        (low_m_chi2,low_m_p,low_m_dof,low_m_expected_table)=stats.chi2_contingency(low_m_table)
        #higher than threshold
        high_m_table=pd.crosstab(df.loc[:,'Microglia-specific'],df.loc[:,'high_dN/dS'])
        (high_m_OR,high_m_fisher_p)=stats.fisher_exact(high_m_table)
        (high_m_chi2,high_m_p,high_m_dof,high_m_expected_table)=stats.chi2_contingency(high_m_table)
        
        table_df=table_df.append({'GO':GO,'num_genes':num_genes,
                                  'n_med':n_med,'low_n_chi2':low_n_chi2,'low_n_p':low_n_p,
                                  'low_n_OR':low_n_OR,'low_n_fisher_p':low_n_fisher_p,
                                  'high_n_chi2':high_n_chi2,'high_n_p':high_n_p,
                                  'high_n_OR':high_n_OR,'high_n_fisher_p':high_n_fisher_p,
                                  'e_med':e_med,'low_e_chi2':low_e_chi2,'low_e_p':low_e_p,
                                  'low_e_OR':low_e_OR,'low_e_fisher_p':low_e_fisher_p,
                                  'high_e_chi2':high_e_chi2,'high_e_p':high_e_p,
                                  'high_e_OR':high_e_OR,'high_e_fisher_p':high_e_fisher_p,
                                  'o_med':o_med,'low_o_chi2':low_o_chi2,'low_o_p':low_o_p,
                                  'low_o_OR':low_o_OR,'low_o_fisher_p':low_o_fisher_p,
                                  'high_o_chi2':high_o_chi2,'high_o_p':high_o_p,
                                  'high_o_OR':high_o_OR,'high_o_fisher_p':high_o_fisher_p,
                                  'a_med':a_med,'low_a_chi2':low_a_chi2,'low_a_p':low_a_p,
                                  'low_a_OR':low_a_OR,'low_a_fisher_p':low_a_fisher_p,
                                  'high_a_chi2':high_a_chi2,'high_a_p':high_a_p,
                                  'high_a_OR':high_a_OR,'high_a_fisher_p':high_a_fisher_p,
                                  'm_med':m_med,'low_m_chi2':low_m_chi2,'low_m_p':low_m_p,
                                  'low_m_OR':low_m_OR,'low_m_fisher_p':low_m_fisher_p,
                                  'high_m_chi2':high_m_chi2,'high_m_p':high_m_p,
                                  'high_m_OR':high_m_OR,'high_m_fisher_p':high_m_fisher_p},ignore_index=True)
    except ValueError:
        pass
table_df

,GO,num_genes,n_med,low_n_chi2,low_n_p,low_n_OR,low_n_fisher_p,high_n_chi2,high_n_p,high_n_OR,...,high_a_fisher_p,m_med,low_m_chi2,low_m_p,low_m_OR,low_m_fisher_p,high_m_chi2,high_m_p,high_m_OR,high_m_fisher_p
0,regulation_of_cell_size,84,0.061608,1.849678,1.738208e-01,2.074074,1.665159e-01,1.128458,2.881045e-01,0.328125,...,0.613223,0.139371,1.552952,2.127002e-01,0.296552,1.955069e-01,1.064189,3.022617e-01,3.095238,1.497642e-01
1,membrane_depolarization,43,0.051598,6.392083,1.146305e-02,7.500000,5.746084e-03,1.840827,1.748540e-01,0.145833,...,0.316425,0.137043,3.220588,7.271750e-02,0.000000,3.209472e-02,1.213389,2.706620e-01,5.666667,1.354989e-01
2,membrane_hyperpolarization,11,0.040020,0.076389,7.822521e-01,3.000000,5.670996e-01,0.860648,3.535574e-01,0.000000,...,0.345455,0.379782,0.009167,9.237249e-01,0.000000,4.545455e-01,0.748611,3.869163e-01,inf,1.818182e-01
3,regulation_of_membrane_potential,227,0.051006,5.320876,2.107142e-02,1.924889,1.684668e-02,6.116749,1.339069e-02,0.233645,...,0.747858,0.104608,8.349285,3.858391e-03,0.284203,2.819217e-03,4.907043,2.674740e-02,3.442308,1.967194e-02
4,signal transduction,1682,0.064512,54.194818,1.815650e-13,2.364138,4.057052e-13,45.036932,1.933532e-11,0.329058,...,0.028406,0.140850,44.065940,3.174978e-11,0.395551,5.525724e-12,91.819135,9.497169e-22,3.218968,2.048995e-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,cytoskeleton-dependent intracellular transport,39,0.078257,1.053241,3.047619e-01,3.000000,2.552356e-01,0.602206,4.377376e-01,0.250000,...,1.000000,0.149288,0.280893,5.961166e-01,0.000000,5.557501e-01,0.000410,9.838518e-01,2.583333,4.361649e-01
63,protein folding,24,0.079997,0.259304,6.105989e-01,1.250000,1.000000e+00,0.630252,4.272629e-01,0.238095,...,0.249012,0.226065,0.259304,6.105989e-01,0.000000,5.296443e-01,1.235294,2.663799e-01,4.333333,1.670481e-01
64,vacuolar transport,22,0.082688,0.071837,7.886810e-01,0.800000,1.000000e+00,0.367586,5.443233e-01,1.083333,...,1.000000,0.057516,0.009860,9.209023e-01,1.600000,1.000000e+00,0.072652,7.875136e-01,1.875000,1.000000e+00
65,plasma membrane organization,44,0.053500,2.234921,1.349236e-01,3.750000,1.317389e-01,0.676923,4.106483e-01,0.000000,...,0.267158,0.121203,1.766249,1.838470e-01,0.000000,1.547337e-01,1.282591,2.574178e-01,5.833333,1.299954e-01


In [79]:
table_df.to_excel('../results/GO_contingency_stats.xlsx')

In [86]:
df = gene_info_df
for GO in go_list:
    GO_df = df.loc[df[GO] == True][['Neuron-specific','Endothelia-specific','Astrocyte-specific','Microglia-specific','Oligodendrocyte-specific','avg_dNdS']].copy(deep=True)

    GO_df['Neuron-specific']=GO_df['Neuron-specific'].map({'In Neuron':1, 'NOT In Neuron':np.nan})
    GO_df['Endothelia-specific']=GO_df['Endothelia-specific'].map({'In Endothelia':1, 'NOT In Endothelia':np.nan})
    GO_df['Astrocyte-specific']=GO_df['Astrocyte-specific'].map({'In Astrocyte':1, 'NOT In Astrocyte':np.nan})
    GO_df['Microglia-specific']=GO_df['Microglia-specific'].map({'In Microglia':1, 'NOT In Microglia':np.nan})
    GO_df['Oligodendrocyte-specific']=GO_df['Oligodendrocyte-specific'].map({'In Oligodendrocyte':1, 'NOT In Oligodendrocyte':np.nan})

    GO_df['Neuron-specific_dN/dS'] = GO_df['avg_dNdS']*GO_df['Neuron-specific']
    GO_df['Endothelia-specific_dN/dS'] = GO_df['avg_dNdS']*GO_df['Endothelia-specific']
    GO_df['Astrocyte-specific_dN/dS'] = GO_df['avg_dNdS']*GO_df['Astrocyte-specific']
    GO_df['Microglia-specific_dN/dS'] = GO_df['avg_dNdS']*GO_df['Microglia-specific']
    GO_df['Oligodendrocyte-specific_dN/dS'] = GO_df['avg_dNdS']*GO_df['Oligodendrocyte-specific']

    GO_df.drop(GO_df.columns[0:5],axis=1,inplace=True)

    GO_avg = GO_df['avg_dNdS'].mean(axis=0)
    GO_med = GO_df['avg_dNdS'].median(axis=0)

    # get numpy array of each cell type's dN/dS scores without NaN
    N_arr = GO_df['Neuron-specific_dN/dS'].dropna().values
    E_arr = GO_df['Endothelia-specific_dN/dS'].dropna().values
    A_arr = GO_df['Astrocyte-specific_dN/dS'].dropna().values
    M_arr = GO_df['Microglia-specific_dN/dS'].dropna().values
    O_arr = GO_df['Oligodendrocyte-specific_dN/dS'].dropna().values

    # Creating a figure 
    fig = plt.figure(figsize=(7.5,10)) # Size of a letter size paper
    fig.suptitle('Five Celltypes Distribution of Average dN/dS Scores of Genes Related to'+"\n"+GO, fontsize=14)

    # Setting subplot space
    grid = plt.GridSpec(nrows=15,ncols=2)
    grid.update(wspace=0.3, hspace=0.3)

    # The subplot for distribution histogram 
    distr_plot = fig.add_subplot(grid[0:4,:])
    #distr_hist = dist_GO_df.plot(ax=distr_plot, kind='hist',alpha=0.3,bins=np.logspace(np.log10(0.001),np.log10(10), 100), logx=True, color=['royalblue','orange','lightgreen'])

    # Set up the bins for log scale x-axis, and get the centers
    bins=np.logspace(np.log10(0.001),np.log10(10), 100) # pizza
    bins_cntr = (bins[1:] + bins[:-1]) / 2

    # Distribution Histograms of the three cell types
    countsN, bin_edgesN, ignoredN = distr_plot.hist(N_arr, bins, histtype='stepfilled', alpha=0.2, 
                                               color='tab:red')
    countsE, bin_edgesE, ignoredE = distr_plot.hist(E_arr, bins, histtype='stepfilled', alpha=0.2, 
                                               color='tab:orange')
    countsA, bin_edgesA, ignoredA = distr_plot.hist(A_arr, bins, histtype='stepfilled', alpha=0.2, 
                                               color='lime')
    countsM, bin_edgesM, ignoredM = distr_plot.hist(M_arr, bins, histtype='stepfilled', alpha=0.2, 
                                               color='deepskyblue')
    countsO, bin_edgesO, ignoredO = distr_plot.hist(O_arr, bins, histtype='stepfilled', alpha=0.2, 
                                               color='aquamarine')

    
    # Log-normal Curve for Neuron
    try:
        # calculate area of histograms (area under PDF should be 1)
        area_histN = ((bin_edgesN[1:] - bin_edgesN[:-1]) * countsN).sum()
        # Fitting log normal to cell type specific dN/dS of this GO
        #shapeN, locN, scaleN = stats.lognorm.fit(N_arr, floc=0)
        shapeN, locN, scaleN = stats.lognorm.fit(N_arr)
        # pdf-values using cdf 
        N_fit_log_cntr_ = stats.lognorm.cdf(bins, shapeN, loc=locN, scale=scaleN)
        N_fit_log_cntr = np.diff(N_fit_log_cntr_)
        # plot fitted and scaled PDFs into histogram
        distr_plot.plot(bins_cntr, N_fit_log_cntr * countsN.sum(),color='tab:red',ls='-', 
                        label='Neuron(med={0:.3f})'.format(np.median(N_arr)), linewidth=2)
    except ValueError:
        pass

    # Log-normal Curve for Endothelial cells
    try:
        area_histE = ((bin_edgesE[1:] - bin_edgesE[:-1]) * countsE).sum()
        #shapeE, locE, scaleE = stats.lognorm.fit(E_arr, floc=0)
        shapeE, locE, scaleE = stats.lognorm.fit(E_arr)
        E_fit_log_cntr_ = stats.lognorm.cdf(bins, shapeE, loc=locE, scale=scaleE)
        E_fit_log_cntr = np.diff(E_fit_log_cntr_)
        distr_plot.plot(bins_cntr, E_fit_log_cntr * countsE.sum(),color='tab:orange',ls='-', 
                        label='Endothelia(med={0:.3f})'.format(np.median(E_arr)), linewidth=2)
    except ValueError:
         pass

    # Log-normal Curve for Microglia Cells
    try: 
        area_histM = ((bin_edgesM[1:] - bin_edgesM[:-1]) * countsM).sum()
        #shapeM, locM, scaleM = stats.lognorm.fit(M_arr, floc=0)
        shapeM, locM, scaleM = stats.lognorm.fit(M_arr)
        M_fit_log_cntr_ = stats.lognorm.cdf(bins, shapeM, loc=locM, scale=scaleM)
        M_fit_log_cntr = np.diff(M_fit_log_cntr_)
        distr_plot.plot(bins_cntr, M_fit_log_cntr * countsM.sum(),color='deepskyblue',ls='-', 
                        label='Microglia(med={0:.3f})'.format(np.median(M_arr)), linewidth=2)
    except ValueError:
        pass
    
    # Log-normal Curve for Oligodendrocyte Cells
    try: 
        area_histO = ((bin_edgesO[1:] - bin_edgesO[:-1]) * countsO).sum()
        #shapeO, locO, scaleO = stats.lognorm.fit(O_arr, floc=0)
        shapeO, locO, scaleO = stats.lognorm.fit(O_arr)
        O_fit_log_cntr_ = stats.lognorm.cdf(bins, shapeO, loc=locO, scale=scaleO)
        O_fit_log_cntr = np.diff(O_fit_log_cntr_)
        distr_plot.plot(bins_cntr, O_fit_log_cntr * countsO.sum(),color='aquamarine',ls='-', 
                        label='Oligodendrocyte(med={0:.3f})'.format(np.median(O_arr)), linewidth=2)
    except ValueError:
        pass
    
    # Log-normal Curve for Astrocyte Cells
    try: 
        area_histA = ((bin_edgesA[1:] - bin_edgesA[:-1]) * countsA).sum()
        #shapeA, locA, scaleA = stats.lognorm.fit(A_arr, floc=0)
        shapeA, locA, scaleA = stats.lognorm.fit(A_arr)
        A_fit_log_cntr_ = stats.lognorm.cdf(bins, shapeA, loc=locA, scale=scaleA)
        A_fit_log_cntr = np.diff(A_fit_log_cntr_)
        distr_plot.plot(bins_cntr, A_fit_log_cntr * countsA.sum(),color='lime',ls='-', 
                        label='Astrocyte(med={0:.3f})'.format(np.median(A_arr)), linewidth=2)
    except ValueError:
        pass

    # # If I want to include a shadow of all genes' distribution 
    # #dist_df.plot(ax=distr_plot,kind='hist',alpha=0.2,bins=bins, logx=True, color='beige')

    distr_plot.axvline(med, color='black', linestyle='-', label='All Brain Genes Median')
    distr_plot.axvline(high_cutoff, color='silver', linestyle='--', label='Quantile Cuttoffs')
    distr_plot.axvline(low_cutoff, color='silver', linestyle='--')
    distr_plot.axvline(GO_med, color='chocolate', linestyle='-.', label='Median of this GO')

    distr_plot.set_xlabel(xlabel='dN/dS')
    distr_plot.set_ylabel(ylabel='number of genes')
    distr_plot.set_xscale('log')
    distr_plot.legend(loc='best')

    # The contingency tables and the statisitcs tables 
    try:
        tab_plot(fig,df,GO)
#         Nchi2_num=tab_plot.Nchi2_num
#         Nchi2_num=f"{Nchi2_num:.3f}"
        Np_num=tab_plot.Np_num
        Np_num=f"{Np_num:.5f}"
    except IndexError:
        pass
    except ValueError:
        pass
    
    plt.subplots_adjust(right=0.98)
#     if to rank the pdfs with stats 
#     fig.savefig('../figures/contingency_analysis/'+Np_num+' '+GO+' contingency analysis.pdf')
    fig.savefig('../figures/contingency_analysis/'+GO+'.pdf')

    plt.close()

Text(0.5, 0.98, 'Five Celltypes Distribution of Average dN/dS Scores of Genes Related to\nregulation_of_cell_size')

Text(0.5, 0, 'dN/dS')

Text(0, 0.5, 'number of genes')

Text(0.5, 0.98, 'Five Celltypes Distribution of Average dN/dS Scores of Genes Related to\nmembrane_depolarization')

Text(0.5, 0, 'dN/dS')

...

Text(0.5, 0, 'dN/dS')

Text(0, 0.5, 'number of genes')

Text(0.5, 0.98, 'Five Celltypes Distribution of Average dN/dS Scores of Genes Related to\nphotosynthesis')

Text(0.5, 0, 'dN/dS')

Text(0, 0.5, 'number of genes')